In [1]:
# ! pip install pandas
# ! pip install ftfy regex tqdm
# ! pip install git+https://github.com/openai/CLIP.git

In [2]:
import pandas as pd
import os

C:\Users\ribal\AppData\Local\Temp\ipykernel_12724\3026406390.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
data_dir = "data/"
images_dir = "Food Images\Food Images"
csv_file = "Food Ingredients and Recipe Dataset with Image Name Mapping.csv"

In [4]:
df = pd.read_csv(os.path.join(data_dir, csv_file))

In [5]:
df.columns

Index(['Unnamed: 0', 'Title', 'Ingredients', 'Instructions', 'Image_Name',
       'Cleaned_Ingredients'],
      dtype='object')

In [6]:
names = df["Image_Name"]
len(names)

13501

In [7]:
seen = set()
doubles = []
for x in names:
    if x not in seen:
        seen.add(x)
    else:
        doubles.append(x)

In [8]:
len(doubles)

29

In [9]:
df_cleared = df[df['Image_Name']!='#NAME?']
# df_cleared.iloc[4000]
# len(df_cleared)

In [ ]:
images = os.listdir(os.path.join(data_dir, images_dir))
len(images)

['-bloody-mary-tomato-toast-with-celery-and-horseradish-56389813.jpg',
 '-burnt-carrots-and-parsnips-56390131.jpg',
 '-candy-corn-frozen-citrus-cream-pops-368770.jpg',
 '-candy-corn-pumpkin-blondies-51254510.jpg',
 '-carbonnade-a-la-flamande-short-ribs-358557.jpg',
 '-chickpea-barley-and-feta-salad-51239040.jpg',
 '-chickpea-pancakes-with-leeks-squash-and-yogurt-51260630.jpg',
 '-cod-with-mussels-chorizo-fried-croutons-and-saffron-mayonnaise-355204.jpg',
 '-em-ba-em-s-ultimate-lobster-rolls-51169080.jpg',
 '-em-gourmet-live-em-s-first-birthday-cake-367789.jpg',
 '-em-polow-em-persian-rice-with-pistachios-and-dill-352469.jpg',
 '-em-stracciatella-tortoni-em-cake-with-espresso-fudge-sauce-242605.jpg',
 '-fried-chicken-51238060.jpg',
 '-fritto-misto-51252640.jpg',
 '-halibut-confit-with-leeks-coriander-and-lemon-51252690.jpg',
 '-hazelnut-butter-and-coffee-meringues-51260360.jpg',
 '-lentils-with-cucumbers-chard-and-poached-egg-51260640.jpg',
 '-like-a-caesar-235836.jpg',
 '-pistachio-cru

In [11]:
set(images)
len(images)

13582

#### TEST CLIP

In [12]:
import torch
import clip
from PIL import Image

In [13]:
models = clip.available_models()
models

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [14]:
model_name = 'RN50x4'
my_model, preprocess = clip.load(model_name)
# print("NUM_PARAMS", sum(p.numel() for p in my_model.parameters() if p.requires_grad))

In [15]:
print(my_model)

CLIP(
  (visual): ModifiedResNet(
    (conv1): Conv2d(3, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU(inplace=True)
    (conv2): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2): ReLU(inplace=True)
    (conv3): Conv2d(40, 80, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn3): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu3): ReLU(inplace=True)
    (avgpool): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(80, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
     

In [16]:
idx = 0
ingredients, label = eval(df.loc[idx, "Cleaned_Ingredients"]), df.loc[idx, "Image_Name"]
ingredients = " ".join(ingredients)
print(len(ingredients.split()))
ingredients, label

133


('1 (3½–4-lb.) whole chicken 2¾ tsp. kosher salt, divided, plus more 2 small acorn squash (about 3 lb. total) 2 Tbsp. finely chopped sage 1 Tbsp. finely chopped rosemary 6 Tbsp. unsalted butter, melted, plus 3 Tbsp. room temperature ¼ tsp. ground allspice Pinch of crushed red pepper flakes Freshly ground black pepper ⅓ loaf good-quality sturdy white bread, torn into 1" pieces (about 2½ cups) 2 medium apples (such as Gala or Pink Lady; about 14 oz. total), cored, cut into 1" pieces 2 Tbsp. extra-virgin olive oil ½ small red onion, thinly sliced 3 Tbsp. apple cider vinegar 1 Tbsp. white miso ¼ cup all-purpose flour 2 Tbsp. unsalted butter, room temperature ¼ cup dry white wine 2 cups unsalted chicken broth 2 tsp. white miso Kosher salt freshly ground pepper',
 'miso-butter-roast-chicken-acorn-squash-panzanella')

#### TEXT to IMAGE

In [110]:
#ingredients_focus_stop_words
STOP_WORDS = [
    # Quantities and Measurements
    "tsp", "teaspoon", "teaspoons", "tbsp", "tablespoon", "tablespoons",
    "c", "cup", "cups", "ml", "milliliter", "milliliters", "l", "liter", "liters",
    "g", "gram", "grams", "kg", "kilogram", "kilograms", "oz", "ounce", "ounces",
    "lb", "pound", "pounds", "inch", "inches", "cm", "centimeter", "centimeters",
    "mm", "millimeter", "millimeters", "gal", "gallon", "gallons", "qt", "quart", "quarts",
    "pt", "pint", "pints", "fl", "fluid", "pinch", "pinches", "slice", "slices","chucks",
    "package", "packages", "pack", "packs", "piece", "pieces", "bunch", "bunches",
    "whole", "half", "halved", "total", "serving", "part", "parts", "quartered",
    "sliced", "diced", "chopped", "minced", "crushed", "grated", "shredded",
    "small", "medium", "large",
    "cupg", "tbspg", "tbspml", "lbg", "size", "cupsml", "cupsg", "cupskg", "ozg",
    
    # Common Preparatory Terms
    "finely", "roughly", "thinly", "thickly", "coarsely", "freshly", "fresh", "dried",
    "peeled", "trimmed", "cored", "pitted", "melted", "softened", "patted", "packed", "tightly", 
    "divided", "plus", "more", "less", "optional", "needed", "to", "taste", "ground", "removed",  
    
    # Common Ingredients that might be too generic and does not show in image
    "salt", "salted", "unsalted", "pepper", "water", "oil", "olive", "vegetable", "sugar", "brown",
    "sweetened", "unsweetened", "extravirgin", "ripe", 
    "flour", "all-purpose flour", "butter", "fine", "sea",
    
    # Others
    "about", "approximately", "or", "and", "quality", "goodquality", "good", "tasty", "temperature", "room",
    "of", "into", "such", "as", "also", "a", "only", "in", "below", "above", "method", "see", "on", "between", "depending",
    "if", "you", "have", "one", "use", "no", "with", "from", "kosher", "dry", "for", "intact", "cut", "ask", "your",
    "butcher", "too", "it", "can", "other", "long", "the", "too", "note", "slicedsee", "blocks", "extrafirm", "yellow",
    "reserve", "raw", "granulated", "unflavored", "seperated", "pan", "dark", "market", "fancy", "well", "buy", "go", "add", "thin", "by", "leaving", "all",
    "bite", "bitesize", "but", "want", "out", "some", "need", "internet" "throw", "number", "up",
]

In [111]:
import re

def text_preprocess(text):
    # Remove non-alphabet symbols
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    filtered_words = [word.lower() for word in text.split() if word.lower() not in STOP_WORDS]
    
    # Join the filtered words back into a string
    cleaned_text = ' '.join(set(filtered_words))
    
    return cleaned_text

In [112]:
ingredients, text_preprocess(ingredients)

('12 medium to large fresh poblano chiles (2 1/4 lb) 1 (14- to 16-oz) can whole tomatoes including juice 3 large garlic cloves, chopped 1/4 cup chopped fresh cilantro 1/4 teaspoon sugar 1/4 teaspoon salt, or to taste 2 tablespoons olive oil 1 cup heavy cream 2 teaspoons dried epazote* (optional) 8 oz soft mild goat cheese 1/4 teaspoon salt, or to taste 4 (3/4-lb) bunches spinach, coarse stems discarded 2 tablespoons olive oil 1/4 cup raisins 1/4 teaspoon salt, or to taste 3 tablespoons olive oil 6 (6-inch) corn tortillas, halved 1 cup cooked black beans, rinsed and drained if canned Garnish: toasted pine nuts a 2- to 2 1/2-quart shallow flameproof casserole dish (about 2 inches deep; not glass)',
 'garlic cloves chiles not glass mild canned nuts cooked raisins rinsed poblano drained black flameproof spinach corn epazote discarded cream including coarse tortillas cilantro pine stems soft toasted shallow cheese heavy tomatoes goat beans juice casserole garnish deep dish')

In [115]:
device = "cuda" if torch.cuda.is_available() else "cpu"
scores = {}

# text = clip.tokenize(text_preprocess(ingredients)).to(device)
# text_features = my_model.encode_text(text)
idx = 50
test_image = images[idx]
image_path = os.path.join(data_dir, images_dir, test_image)
image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)

# for image_name in images[:50]:

#     with torch.no_grad():
#         image_features = my_model.encode_image(image)
#         logits_per_image, logits_per_text = my_model(image, text_features)
#         probs = logits_per_image.softmax(dim=-1).cpu().numpy()

#     print("Label probs:", probs)  # prints: [[0.9927937  0.00421068 0.00299572]]

# Calculate features
text_inputs = []
labels = []
for idx in range(len(df_cleared)):
    ingredients, label = eval(df_cleared.iloc[idx].Cleaned_Ingredients), df_cleared.iloc[idx].Image_Name
    ingredients = " ".join(ingredients)
    text_inputs.append(text_preprocess(ingredients))
    labels.append(label)

text_inputs = torch.cat([clip.tokenize(row) for row in text_inputs]).to(device)

RuntimeError: Input soy is garlic box unseasoned lengthwise ginger regular chicken hot cloves then coriander packaged shallot flakes drained halfmoons tomato black table grater cashews stirfry vermouth moons marinated olives strips grape sprigs that bell yukon inchthick broth sesame thyme white cinnamon roasted cumin stemmed sauce wine multibranched toasted sundried yogurt onions sriracha tomatoes beans squash vinegar gold turmeric glutenfree red onion breast green through holes pickled rice potato lowfat skinless peanuts plain globe rosemary scallions concern boneless is too long for context length 77

In [ ]:
with torch.no_grad():
    image_features = my_model.encode_image(image)
    text_features = my_model.encode_text(text_inputs)

# Pick the top 5 most similar labels for the image
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
values, indices = similarity[0].topk(5)

# Print the result
print("\nTop predictions:\n")
for value, index in zip(values, indices):
    print(f"{labels[index]:>16s}: {100 * value.item():.2f}%")

RuntimeError: The size of tensor a (300) must match the size of tensor b (77) at non-singleton dimension 1

13470

In [ ]:
ingredients, label = eval(df_cleared.loc[idx, "Cleaned_Ingredients"]), df_cleared.loc[idx, "Image_Name"]

KeyError: 3441

- scp local_dir ribal.teeny@machine:~remote_dir

in host:
- mv remote_dir /Data
- ln-s /Data/mydata project_dir/mydata